In [1]:
import util
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pystan

# Data Loading

We are using MNIST data for this.

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [3]:
X = mnist.train._images.reshape(55000,28,28)
Y = mnist.train._labels
index = np.arange(55000)
np.random.shuffle(index)

bf = util.BatchFeeder(X[index[:54000]], Y[index[:54000]])
valid_data = (X[index[54000:]], Y[index[54000:]])

# Building a CNN model
We use two layer CNN with 16 and 32 kenerls of sizes 3 by 3, followed by a dense layer with 10 hidden units.

In [ ]:
def model(D, N):
    D = D*1.0
    N = N*1.0
    
    #resetting parameters
    util.reset_graph()
    
    # Input and output
    # Use Placeholder in tf for input and output.
    x_tf = tf.placeholder(tf.float64, [None, wine_itr.d])  # data matrix
    y_tf = tf.placeholder(tf.float64, [None, 1]) # target matrix
    lambda_tf = tf.Variable(1.0, name="h", dtype=tf.float64) #tf.Variable(1.0, name="h")
    learning_rate = tf.placeholder(tf.float64)
    
    # Single fully connected layer
    # Loads in the noise + optimal weights calculated above as a starting point
    W_tf = tf.Variable(W, name="w") # weight matrix
    dense = tf.matmul(x_tf, W_tf) # fully connected multiplication
    
    # Objective function. This needs to be absolutely correct.
    cost = tf.reduce_mean(tf.square(tf.subtract(y_tf, dense)))/2.0\
        +lambda_tf/2.0*tf.reduce_sum(tf.square(W_tf))/N\
        -D/2.0*tf.log(lambda_tf)/N
    
    # Optimization operations
    # SGD of learning rate 1 is used because we can manually 
    # multiply gradients with custom learning rate later.
    optimizer_sgd = tf.train.GradientDescentOptimizer(learning_rate)
    
    # Get trainable variables
    tvars = tf.trainable_variables()
    wvars = [i for i in tvars if i._variable.name[0] == "w"]
    hvars = [i for i in tvars if i._variable.name[0] == "h"]
    
    # Get gradients with respect to parameters
    w_gradients = optimizer_sgd.compute_gradients(cost, wvars)
    h_gradients = optimizer_sgd.compute_gradients(cost, hvars)

    # Learing rates holder. This is a tuple of placeholders that
    # match the size of the parameters in the variable holders.
    precision_holders = tuple(tf.placeholder(tf.float64, [np.prod([i.value for i in g[0].shape]),\
                        np.prod([i.value for i in g[0].shape])]) for g in w_gradients)
    
    # Calculating a sgd step given prescition matrix.
    sgd_steps = [(tf.matmul(precision_holders[i], w_gradients[i][0]),\
                           w_gradients[i][1]) for i in range(len(w_gradients))]
    
    # Applying it to current weights.
    train_w = optimizer_sgd.apply_gradients(sgd_steps)
    train_h = optimizer_sgd.apply_gradients(h_gradients)
    
    return dict(
        x = x_tf,
        y = y_tf,
        _lambda = lambda_tf,
        lr = learning_rate,
        w_gradients = w_gradients,
        h_gradients = h_gradients,
        train_w = train_w,
        train_h = train_h,
        current_w = W_tf,
        precision_matrix = precision_holders
    )